In [1]:
library(MASS) # required for PCAtest
library(Seurat)
library(dplyr)
library(tidyverse)
library(ICtest) # does bootstrapped analysis of significance of PCA for ICA https://cran.r-project.org/web/packages/ICtest/vignettes/PCA.html
library(stats)
# this works in Rstudio but on the notebooks it spits out a 
#library(RGCCA) # regularized CCA vignette here:
# can give significances of individual parts of compenents (I think)
library(PCAtools)
library(pheatmap)
#if (!requireNamespace('BiocManager', quietly = TRUE))
#  install.packages('BiocManager')
#BiocManager::install('PCAtools')
library(loadings)
# statistical singificnace from boots trap for PCA https://peerj.com/articles/12967.pdf
library(PCAtest) # devtools::install_github("arleyc/PCAtest")
# avoids axis inversion "In addition, the problem of ‘axis reflection’ (i.e., the arbitrary permutation of
#signs among loadings and PC scores), which is well known in the literature (Jackson,
#1995; Mehlman, Shepherd & Kelt, 1995; Peres-Neto, Jackson & Somers, 2003, 2005), is
#effectively avoided with these two statistics as originally implemented by Vieira (2012) and
#in the R package introduced here"
library(RColorBrewer)
# another potential contender: https://rdrr.io/github/ucsf-ferguson-lab/syndRomics/man/boot_pca_sample.prcomp.html
# also claims to avoid axis inversion

# this can compute significant loadings on PCA
# library(loadings)
# this functions pca_loading() take prcomp as input
# https://cran.r-project.org/web/packages/loadings/loadings.pdf

# Yamamoto, H., Fujimori, T., Sato, H., Ishikawa, G., Kami, K., & Ohashi, Y. (2014). 
# Statistical hypothesis testing of factor loading in principal component analysis and its application 
# to metabolite set enrichment analysis. BMC bioinformatics, 15, 1-9.

Attaching SeuratObject

Attaching sp


Attaching package: 'dplyr'


The following object is masked from 'package:MASS':

    select


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


-- Attaching packages --------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.4.0     v purrr   0.3.4
v tibble  3.1.8     v stringr 1.4.1
v tidyr   1.2.1     v forcats 0.5.2
v readr   2.1.3     

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
x dplyr::select() masks MASS::select()

Loading required package: JADE

Loading required package: ICS

Loading required package: mvtnorm

Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2

Loading required package: ggrepel


Attaching package: 'PCAtools'


The following objects are masked from 

In [2]:
# for bootstrapping significance from PCA
# ICtest library
# https://cran.r-project.org/web/packages/ICtest/ICtest.pdf
# https://cran.r-project.org/web/packages/ICtest/index.html
# https://cran.r-project.org/web/packages/ICtest/vignettes/PCA.html
#install.packages('ICtest')

In [3]:
# function for cross validated CCA
#https://rdrr.io/github/giac01/ccatools/man/cca_cv_boot.html
#install.packages('ccatools')

In [4]:
# set paths
# from this pub: https://www.nature.com/articles/s41586-022-04915-7#Sec5
# cell assingments were done with this: 
bugeon.path <- '/space/scratch/Bugeon2022_merfish_ca2plus/'


# Hrvatin, S., Hochbaum, D. R., Nagy, M. A., Cicconet, M., Robertson, K., Cheadle, L., ... & Greenberg, M. E. (2018). 
# Single-cell analysis of experience-dependent transcriptomic states in the mouse visual cortex. Nature neuroscience, 21(1), 120-129.
#From here: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5742025/
hrvatin.path <- '/home/acampbell/test_datasets/Hrvatin2018/'

bahl2022.path <- '/home/acampbell/PavLabEngrams/IEG_gradients/neurestimator_iegs_list.csv'

tasic.path <- '/space/scratch/Tasic2018_GSE115746/'

neurestimator.iegslist.path <- paste(getwd(),'/','neuroestimator_iegs_list.csv',sep='')
neurestimator.iegslist.path

[1] "/home/acampbell/PavLabEngrams/IEG_gradients/neuroestimator_iegs_list.csv"

In [5]:
hg_to_mm <- 'PavLabEngrams/EngramCellClassifier/hg_mm_1to1_ortho_genes_DIOPT-v8.tsv'

In [6]:
getwd()

[1] "/home/acampbell/PavLabEngrams/IEG_gradients"

In [7]:
# functions

pseudocount_log2p1_transform <- function(x, scale_factor = 10^6, UMI.provided = NULL){
  # Almost as Seurat::NormalizeData but we use log2 rather than natural log
  # from here https://satijalab.org/seurat/reference/normalizedata
  if(is.null(UMI.provided)){
    counts <- sum(x)}else{
      counts <- UMI.provided
    }
  x <- (x)/counts # Feature counts for each cell are divided by the total counts for that cell...
  x <- x*scale_factor # and multiplied by the scale.factor. 
  # the we log2 plus 1 rather than natural log plus 1 seurat uses
  return(log2(x+1))
}

pavlab.normalize <- function(df, UMI = NULL, median.scale=FALSE, scaleby = 10000){
  df.cols <- colnames(df)
  df.rows <- rownames(df)
  if(median.scale){ scaleby = median(UMI)}
  if( is.null(UMI)){
    df <- data.frame(apply(df,  MARGIN = 2, pseudocount_log2p1_transform))
  }else{
    #
    df[] <- Map(pseudocount_log2p1_transform, df, scale_factor = scaleby, UMI.provided = UMI)
    
  }
  colnames(df) <- df.cols
  rownames(df)<- df.rows
  return(df)
}

# for finding nth hihgest value, we use it for fincing geens of highest varience
maxN <- function(x, N=2){
    len <- length(x)
    if(N>len){
        warning('N greater than length(x).  Setting N=length(x)')
        N <- length(x)
    }
    sort(x,partial=len-N+1)[len-N+1]
}

# to write pheatmap images to file as a .png
save_pheatmap <- function(x, filename, width=480, height=960) {
   stopifnot(!missing(x))
   stopifnot(!missing(filename))
   png(filename,width = width, height=height)
   grid::grid.newpage()
   grid::grid.draw(x$gtable)
   dev.off()
}

### Gene Lists

I put together several lists of genes.

Bugeon et al., (2022) coppaFISH probes, chosen on basis of being best able to disntinguish cell types in V1.  I also wrote out the order they came in in tPC1 they report in thier publication.  Coppied from figure 5.

From Bahl et al., (2022), not a published paper just a preprint.  Use their IEGs, the 41 thy had before filtering to the most correlated together.  These are just the genes that shoed differntial expression in response to some stimulation paradigm with a fold change of 0.5 across thre three studies they chose.  I did not include their last criteria for gene being correlated to one another because I wanted genes that may be more likely to exhibit gradient effects.

Ensemble protien coding mouse genes with stable transcript IDs and unique mapping.

In [8]:
#gene order of bugeon tPC1
tPC1.bugeon2022 <- c("Pvalb","Slc6a1","Gad1","Lhx6","Serpini1","Tac1","Npy","Cox6a2",
                    "Gabrd","Rgs4","Cort","Prkca","Crhbp","Sst","Satb1","Calb1","Ntng1",
                    "Col25a1","Rasgrf2","Nrn1","Rab3c", "Bcl11b","Cdh13","Neurod6","Enpp2",
                    "Hapln1","Wfs1","Pthlh","Lamp5","Kcnk2","Thsd7a","Aldoc","Grin3a","Plcxd2",
                    "Slc17a8","Plp1","Th","Chodl","Nr4a2","Nos1","Cpne5","Chrm2","Gda",
                    "Nov","Npy2r","Sema3c","Ndnf","Sncg","Kctd12","Pcp4","Calb2","Trp53i11",
                    "Cck","Rgs10","Pde1a","Cryab","Crh","Cadps2","Pnoc","Synpr","Id2","Rgs12",
                    "Snca","Penk","Kit","Cplx2","Reln","Tac2","Htr3a","Vip","Cnr1","Cxcl14")

In [9]:
# named meta for being required to exhibit a 0.5 foldchnage in 2 out of 3 studies from Bahl et al., 2022 in biorvx
# neuroestimator studies are PMID: 31501571, PMID: 29681534, PMID: 24855953
bahl.meta.iegs.df <- read.csv(bahl2022.path)
meta.iegs <- bahl.meta.iegs.df$stimulus_responsive_gene
meta.iegs

[1] "Arc"      "Atf3"     "Btg2"     "Ccdc184"  "Coq10b"   "Crem"    
 [7] "Csrnp1"   "Dusp1"    "Dusp5"    "Egr1"     "Egr2"     "Egr3"    
[13] "Errfi1"   "Fbxo33"   "Fos"      "Fosb"     "Fosl2"    "Gadd45g" 
[19] "Gmeb2"    "Grasp"    "Herpud1"  "Ier2"     "Junb"     "Klf4"    
[25] "Mbnl2"    "Npas4"    "Nr4a1"    "Nr4a2"    "Nr4a3"    "Pcsk1"   
[31] "Peg10"    "Per1"     "Plagl1"   "Ppp1r15a" "Ptgs2"    "Rem2"    
[37] "Rgs2"     "Sertad1"  "Sik1"     "Stard4"   "Tiparp"

In [10]:
# gene names of protien coding genes in mice
prot.coding.genes.ensmlble.df <- read.csv('/home/acampbell/PavLabEngrams/IEG_gradients/unique_stablestranscriptIDs_m39_gene_names.txt')
mouse.prot.coding.genes <- unique(prot.coding.genes.ensmlble.df$Gene.name) # we don't need to map ensemble id's to gene names
print(paste("There are this many unique protien coding gene names in our list:", as.character(length((mouse.prot.coding.genes)), sep='')))
head(prot.coding.genes.ensmlble.df)

[1] "There are this many unique protien coding gene names in our list: 21751"


,Gene.stable.ID,Gene.stable.ID.version,Transcript.stable.ID,Transcript.stable.ID.version,Gene.name
,<chr>,<chr>,<chr>,<chr>,<chr>
1,ENSMUSG00000064341,ENSMUSG00000064341.1,ENSMUST00000082392,ENSMUST00000082392.1,mt-Nd1
2,ENSMUSG00000064345,ENSMUSG00000064345.1,ENSMUST00000082396,ENSMUST00000082396.1,mt-Nd2
3,ENSMUSG00000064351,ENSMUSG00000064351.1,ENSMUST00000082402,ENSMUST00000082402.1,mt-Co1
4,ENSMUSG00000064354,ENSMUSG00000064354.1,ENSMUST00000082405,ENSMUST00000082405.1,mt-Co2
5,ENSMUSG00000064356,ENSMUSG00000064356.1,ENSMUST00000082407,ENSMUST00000082407.1,mt-Atp8
6,ENSMUSG00000064357,ENSMUSG00000064357.1,ENSMUST00000082408,ENSMUST00000082408.1,mt-Atp6


In [ ]:
bugeon2022.genes <- as.list(read.table(paste(bugeon.path,'genes.names.txt',sep ='')))
bugeon2022.genes <- bugeon2022.genes$V1
print(length(bugeon2022.genes))
bugeon2022.genes
# spearmen rank correlation between Bugeon genes and PC1 ranking of them from Hrvatin or Taisic

## Datasets we will use

Hrvatin et al., (2018) Dark house mice then exposed to light at 0h (no exposure), 1hr, 4hrs 5hrs (double chekc these time points).

Tasic et al., (2018) COntains scRNA-seq from allen brain institute of multiple cortical regions including mouse V1.   Comes with detailed metadata that Bugeon based their classification hierachy on.



In [11]:
hrvatin2018.meta <- read.csv(paste(hrvatin.path,'GSE102827_cell_type_assignments.csv.gz', sep = ''))
head(hrvatin2018.meta)

,X,stim,sample,maintype,celltype,subtype
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,x2_35_0_bc0013,0h,B1_1_0h,Excitatory,ExcL4,ExcL4_3
2,x2_35_0_bc0014,0h,B1_1_0h,Excitatory,ExcL5_3,NA
3,x2_35_0_bc0016,0h,B1_1_0h,Excitatory,ExcL5_2,NA
4,x2_35_0_bc0017,0h,B1_1_0h,Excitatory,ExcL6,NA
5,x2_35_0_bc0018,0h,B1_1_0h,Excitatory,ExcL5_3,NA
6,x2_35_0_bc0019,0h,B1_1_0h,Excitatory,ExcL23,ExcL23_1


In [12]:
table(hrvatin2018.meta$celltype)


     Astro     Endo_1     Endo_2     ExcL23      ExcL4    ExcL5_1    ExcL5_2 
      7039       3327        123       2963       3198       1976        410 
   ExcL5_3      ExcL6        Hip    Int_Cck    Int_Npy     Int_Pv  Int_Sst_1 
      1407       3276        137         91        137        227        174 
 Int_Sst_2    Int_Vip Macrophage    Micro_1    Micro_2      OPC_1      OPC_2 
       181        126        537        549       9609       1725        101 
    Olig_1     Olig_2     Olig_3     Olig_4     Olig_5     Olig_6     Olig_7 
       610        964        846       4047        747        786        630 
  Pericyte        RSP       SM_1       SM_2        Sub 
       782        420        323        298        500 

In [13]:
# cell types in figure 1f Int_Cck, Int_Npy, Int_Pv, Int_Sst_1, Int_Sst_2, Int_Vip

In [ ]:
hrvatin2018.counts <- read.csv(paste(hrvatin.path, 'GSE102827_merged_all_raw.csv.gz',sep=''))

In [ ]:
dim(hrvatin2018.counts)

In [ ]:
rownames(hrvatin2018.counts) <- hrvatin2018.counts$X

In [ ]:
# sum(is.na(hrvatin2018.counts)) #this will show no na's
hrvatin2018.counts <- hrvatin2018.counts[,2:dim(hrvatin2018.counts)[2]]

In [ ]:
head(hrvatin2018.counts)

In [ ]:
hrvatin2018.meta$umi <- colSums(hrvatin2018.counts)
print(hrvatin2018.meta$umi[1:10])
hrvatin2018.umi.medianforscalefactor <- median(hrvatin2018.meta$umi)
print(hrvatin2018.umi.medianforscalefactor)

In [ ]:
hrvatin.inhibitory.idx <- grep('Int', hrvatin2018.meta$celltype)
length(hrvatin.inhibitory.idx)

In [ ]:
bugeon2022.genes

In [ ]:
tPC1.bugeon2022 %in% bugeon2022.genes$V1

In [ ]:
tPC1.bugeon2022[!(tPC1.bugeon2022 %in% bugeon2022.genes)]

### Hrvatin

Note this data is from dark housed mice exposed ot light for the first time, it's hardly a natural setting.

In [ ]:
hrvatin.counts.inhibitory <- hrvatin2018.counts[,hrvatin.inhibitory.idx]
hrvatin.counts.inhibitory.normed <- pavlab.normalize(hrvatin.counts.inhibitory, 
                                                     UMI=hrvatin2018.meta$umi[hrvatin.inhibitory.idx], 
                                                     scaleby = hrvatin2018.umi.medianforscalefactor)

In [ ]:
hrvatin.counts.inhibitory.normed <- as.data.frame(scale(t(hrvatin.counts.inhibitory.normed)))
rownames(hrvatin.counts.inhibitory.normed) <- colnames(hrvatin.counts.inhibitory)
colnames(hrvatin.counts.inhibitory.normed) <- rownames(hrvatin.counts.inhibitory)

In [ ]:
#GSE115746_accession_table.csv.gz  GSE115746_cells_exon_counts.csv.gz  GSE115746_complete_metadata_28706-cells.csv.gz  GSE115746_controls_exon_counts.csv.gz

In [ ]:
hrvatin.inhib.pca <- pca(hrvatin.counts.inhibitory.normed)

Looks like the activity state of the cells is dominating the data.  There is one PC and no others according to this.

In [ ]:
plot(hrvatin.inhib.pca$variance)

In [ ]:
tPC1_fromrnaseq.hrvatingenes <- data.frame(gene = colnames(hrvatin.counts.inhibitory.normed), tPC1=hrvatin.inhib.pca$rotated$PC1) %>%
    arrange(tPC1)
print(dim(tPC1_fromrnaseq.hrvatingenes ))
head(tPC1_fromrnaseq.hrvatingenes)

In [ ]:
tPC1_fromrnaseq.bugeongenes <- tPC1_fromrnaseq.hrvatingenes[tPC1_fromrnaseq.hrvatingenes$gene %in% bugeon2022.genes,]
tPC1_fromrnaseq.bugeongenes 

In [ ]:
# careful here PCA can flip signs, just make sure Pvalb and Vip are at opposing ends of the rankings
rank.tPC1.from_rnaseq <- c(1:length(tPC1_fromrnaseq.bugeongenes$gene))
rank.tPC1.from_coppaFISH <- c()
for(thisgene in tPC1.bugeon2022){
    position_in_rnaseqPC1 <- match(thisgene, tPC1_fromrnaseq.bugeongenes$gene)
    rank.tPC1.from_coppaFISH <- c(rank.tPC1.from_coppaFISH, position_in_rnaseqPC1 ) # append position in other list
}

In [ ]:
tPC1_fromrnaseq.bugeongenes$rank_in_hrvatin <- rank.tPC1.from_rnaseq
tPC1_fromrnaseq.bugeongenes$rank_in_bugeon <- rank.tPC1.from_coppaFISH
tPC1_fromrnaseq.bugeongenes$gene <- as.factor(tPC1_fromrnaseq.bugeongenes$gene)
rownames(tPC1_fromrnaseq.bugeongenes)

In [ ]:
head(tPC1_fromrnaseq.bugeongenes)

In [ ]:
cor.test( tPC1_fromrnaseq.bugeongenes$rank_in_hrvatin,  tPC1_fromrnaseq.bugeongenes$rank_in_bugeon, 
         method = 'spearman', data = tPC1_fromrnaseq.bugeongenes)

So we have a good correlation when we use all the genes in Hrvatin, rember that this is a highly abnormal dataset,
these mice have been darkhoused for 2 months.  Both resticting it to the Bugeon genes only and computing PCs as well as just the original mapping presserves the rank of the genes in the PC, though admittedly they all have the same sign in the full gene list (i.e. are all positively or negatively loaded).

In [ ]:
# and if we calculate it with just the bugeon genes for the pca?
hrvatin.inhib.pca.restrictedlist <- pca(hrvatin.counts.inhibitory.normed[,bugeon2022.genes])
plot(hrvatin.inhib.pca.restrictedlist$variance)

This looks a little healthier honestly.

In [ ]:
colnames(hrvatin.counts.inhibitory.normed[,bugeon2022.genes])

In [ ]:
tPC1_fromrnaseq.hrvatingenes <- data.frame(gene = colnames(hrvatin.counts.inhibitory.normed[,bugeon2022.genes]),
                                           tPC1 = hrvatin.inhib.pca.restrictedlist$rotated$PC1) %>% arrange(tPC1)
    
print(dim(tPC1_fromrnaseq.hrvatingenes ))
tPC1_fromrnaseq.hrvatingenes

In [ ]:
# careful here PCA can flip signs, just make sure Pvalb and Vip are at opposing ends of the rankings
rank.tPC1.from_rnaseq <- c(1:length(tPC1_fromrnaseq.bugeongenes$gene))
rank.tPC1.from_coppaFISH <- c()
for(thisgene in tPC1.bugeon2022){
    position_in_rnaseqPC1 <- match(thisgene, tPC1_fromrnaseq.bugeongenes$gene)
    rank.tPC1.from_coppaFISH <- c(rank.tPC1.from_coppaFISH, position_in_rnaseqPC1 ) # append position in other list
}

In [ ]:
tPC1_fromrnaseq.bugeongenes$rank_in_hrvatin <- rank.tPC1.from_rnaseq
tPC1_fromrnaseq.bugeongenes$rank_in_bugeon <- rank.tPC1.from_coppaFISH
tPC1_fromrnaseq.bugeongenes$gene <- as.factor(tPC1_fromrnaseq.bugeongenes$gene)
rownames(tPC1_fromrnaseq.bugeongenes)

In [ ]:
head(tPC1_fromrnaseq.bugeongenes)

In [ ]:
cor.test( tPC1_fromrnaseq.bugeongenes$rank_in_hrvatin,  tPC1_fromrnaseq.bugeongenes$rank_in_bugeon, 
         method = 'spearman', data = tPC1_fromrnaseq.bugeongenes)

So the rank correlation is 0.5 between both datasets for the ordering on  PC1 of the gtranscriptomic data in Hrvatin and Bugeon.
with decent significance.  Enough to pass a bonferroni threshold for a number of tests.  You can also note Just by lookuing at the major cell type markers in the PCs that they oppose one another, espescially in the restricted list PC.  This is just from less than 1000 cells between the two datasets as well.

### Tasic data

In [ ]:
list.files(tasic.path)

In [ ]:
tasic2018.meta <- read_csv(paste(tasic.path,'GSE115746_complete_metadata_28706-cells.csv',sep=''))

In [ ]:
head(tasic2018.meta)

In [ ]:
'GSE115746_cells_exon_counts.csv'

In [ ]:
# loading tasic, currently this file is only 78 genes worth of data it has not uploaded properly
tasic.exon.counts <- read.csv(paste(tasic.path,'GSE115746_cells_exon_counts.csv.gz',sep = ''))
print(dim(tasic.exon.counts))
head(tasic.exon.counts)

In [ ]:
rownames(tasic.exon.counts) <- tasic.exon.counts$X
tasic.exon.counts <- tasic.exon.counts[,c(2:dim(tasic.exon.counts)[2])]

In [ ]:
length(colnames(tasic.exon.counts))

Tasic describes no gene filtering beyond alignment, only used exons for UMI counts so we can do that as well I geuss.  Also cells thta did not pass thresholds did not get assigned to a class, subclass, or cluster.

In [ ]:
tasic2018.meta <-read.csv(paste(tasic.path,'GSE115746_complete_metadata_28706-cells.csv', sep = ''))
head(tasic2018.meta)

In [ ]:
#sapply(as.character(jeager2018_meta$source_name), function(y) if (grepl("_F_", y, fixed=TRUE)) "Fos+" else "Fos-"  )
sum(sapply(tasic2018.meta$cell_class, function(x) grepl('GABAergic',x, fixed=TRUE) ))

In [ ]:
# index's to get the V1 gabanergic cells we want
tasic.GABAergic.idx <- sapply(tasic2018.meta$cell_class, function(x) grepl('GABAergic',x, fixed=TRUE) )
tasic.V1.idx <- tasic2018.meta$source_name == 'Primary Visual Cortex (VISp)'
tasic2018v1_inhibitory.meta <- tasic2018.meta[tasic.V1.idx &  tasic.GABAergic.idx,]
dim(tasic2018v1_inhibitory.meta)
#

In [ ]:
# there are some cell types that weren't present in Bugeon probably due to the limits of coppaFISH
# Meis2 and Serpinf1 seem rare, also the unamed cells will be thrown out
table(tasic2018v1_inhibitory.meta$cell_subclass)

In [ ]:
# filtering
gabacellsubclass.keepers <- c("Lamp5","Pvalb","Sncg","Sst","Vip")
tasic2018v1_inhibitory.meta <- tasic2018v1_inhibitory.meta[tasic2018v1_inhibitory.meta$cell_subclass %in% gabacellsubclass.keepers,]
dim(tasic2018v1_inhibitory.meta)

In [ ]:
# filter toe make sure we have matching metadata and count data, the coutsn contain some contorl samples we wont use
v1gaba.cells.idx <- colnames(tasic.exon.counts) %in% tasic2018v1_inhibitory.meta$sample_name
tasic2018.v1_inhibitory.exon.counts <- tasic.exon.counts[,v1gaba.cells.idx]
tasic2018v1_inhibitory.meta <- tasic2018v1_inhibitory.meta[tasic2018v1_inhibitory.meta$sample_name %in% colnames(tasic2018.v1_inhibitory.exon.counts),]
print(dim(tasic2018.v1_inhibitory.exon.counts))
print(dim(tasic2018v1_inhibitory.meta))

In [ ]:
# calculating the scale facotr as the median UMI counts 
tasic.scalefactor.median <- median(colSums(tasic.exon.counts))
tasic.scalefactor.median

Note that because we are not using the controls data it is difficult to give the meta data a UMI comlumn I'd have to assign it to non control cellw hwich seem intermixed in it.  Better to just do filtering on genes afterwards. Also since cells that were not assigned a class or whatever were filtered by the original authors simply by selecting cells which have an assignment we are fitleringout doublets, cells with low reads etc etc

In [ ]:
# note the median it taken from across all cells exonic reads not just inhibitory cells
tasic2018.v1_inhibitory.exon.counts.normed <- pavlab.normalize(tasic2018.v1_inhibitory.exon.counts, scaleby = tasic.scalefactor.median)

In [ ]:
#filter our genes for only protien coding genes
matched.protcoding.genes <- mouse.prot.coding.genes[mouse.prot.coding.genes %in% rownames(tasic2018.v1_inhibitory.exon.counts.normed)]
tasic2018.v1_inhibitory.exon.counts.normed.unfiltered <- tasic2018.v1_inhibitory.exon.counts.normed[matched.protcoding.genes,]
#filter our genes for only protien coding genes
dim(tasic2018.v1_inhibitory.exon.counts.normed.unfiltered)

In [ ]:
# genes in the datasets from our other lsits
print(meta.iegs[meta.iegs  %in% rownames(tasic2018.v1_inhibitory.exon.counts.normed.unfiltered)])
print(bugeon2022.genes[bugeon2022.genes  %in% rownames(tasic2018.v1_inhibitory.exon.counts.normed.unfiltered)])

In [ ]:
# and if we calculate it with just the bugeon genes for the pca?
# this solution need Rfast and matrixStats whihc will not load for some reason
#genevars <- rowVars(as.matrix(tasic2018.v1_inhibitory.exon.counts.normed))
#nth_hihgest.3k <- Rfast::nth(x, 3000, descending = T) # ntoh highest value in this case 3000th hihgest

# alternative solution with customer function and apply()
# originally it was 3k i just left the variable name as is
genevars <- apply(tasic2018.v1_inhibitory.exon.counts.normed.unfiltered, 1, var)
nth_hihgest.3k <- maxN(genevars,4000)

In [ ]:
# checking waht genes survive the thresholding
print("number of genes afer thresholding")
print(sum(genevars>=nth_hihgest.3k))
print("genes from iegs list in set")
print(meta.iegs[meta.iegs  %in% rownames(tasic2018.v1_inhibitory.exon.counts.normed.unfiltered)[genevars>=nth_hihgest.3k]])
print("genes from bugeon in set")
print(bugeon2022.genes[bugeon2022.genes  %in% rownames(tasic2018.v1_inhibitory.exon.counts.normed.unfiltered)[genevars>=nth_hihgest.3k]])

I can live with this, for the IEG figure and actiivty scores we will use the unfiltered data

In [ ]:
print(length(meta.iegs))
meta.iegs

In [ ]:
print(length(bugeon2022.genes))
bugeon2022.genes

In [ ]:
# unfortunately with the full gene set Tasic can't compute the PCs in a reasonable time so I've fitlered it down to the genes that are the
# most hihgly variable in the inhibitory cells spcifically, this unfortunately means we drop out some of the bugeon genes
# the cell type markers remain however
print(length(bugeon2022.genes))
sum(bugeon2022.genes %in% rownames(tasic2018.v1_inhibitory.exon.counts.normed.unfiltered)[genevars>=nth_hihgest.3k])
bugeon2022.genes[bugeon2022.genes %in% rownames(tasic2018.v1_inhibitory.exon.counts.normed.unfiltered)[genevars>=nth_hihgest.3k]]

In [ ]:
#tasic2018.v1_inhibitory.exon.counts.normed <- tasic2018.v1_inhibitory.exon.counts.normed.unfiltered
tasic2018.v1_inhibitory.exon.counts.normed <- tasic2018.v1_inhibitory.exon.counts.normed.unfiltered[genevars>=nth_hihgest.3k,]
bugeongenes.intop3kc <- bugeon2022.genes[bugeon2022.genes %in% rownames(tasic2018.v1_inhibitory.exon.counts.normed)]

In [ ]:
genes <- rownames(tasic2018.v1_inhibitory.exon.counts.normed)
cell.ids <- colnames(tasic2018.v1_inhibitory.exon.counts.normed)

In [ ]:
# quality check to make sure data has not been screwed up
length(genes)
length(cell.ids)

In [ ]:
tasic2018.v1_inhibitory.exon.counts.normed <-as.data.frame(scale(t(tasic2018.v1_inhibitory.exon.counts.normed)))
# after t() the data becomes a matrix and loses its colnames an rownames we need to get them from the og df but row ->cols and vice versa
colnames(tasic2018.v1_inhibitory.exon.counts.normed) <- genes 
rownames(tasic2018.v1_inhibitory.exon.counts.normed) <- cell.ids

In [ ]:
# no NA is good
sum(is.na(tasic2018.v1_inhibitory.exon.counts.normed))

In [ ]:
print(dim(tasic2018.v1_inhibitory.exon.counts.normed))
tasic2018.v1_inhibitory.exon.counts.normed[1:10,1:10]

Now we compute the PC's.

In [ ]:
tasic2018.v1_inhibitory.exon.pca <- pca(tasic2018.v1_inhibitory.exon.counts.normed)
plot(tasic2018.v1_inhibitory.exon.pca$variance)

In [ ]:
tPC1_fromTasic2018 <- data.frame(gene = colnames(tasic2018.v1_inhibitory.exon.counts.normed),
                                           tPC1 = tasic2018.v1_inhibitory.exon.pca$rotated$PC1) %>% arrange(tPC1)
head(tPC1_fromTasic2018)

In [ ]:
tPC1_fromTasic2018.bugeongenesonly <- tPC1_fromTasic2018[tPC1_fromTasic2018$gene %in% bugeon2022.genes,]

print(dim(tPC1_fromTasic2018.bugeongenesonly  ))
tPC1_fromTasic2018.bugeongenesonly

In [ ]:
length(tPC1_fromTasic2018.bugeongenesonly$gene)
duplicated(tPC1_fromTasic2018.bugeongenesonly$gene)

In [ ]:
# careful here PCA can flip signs, just make sure Pvalb and Vip are at opposing ends of the rankings
rank.tPC1.from_rnaseq <- c(1:length(tPC1_fromTasic2018.bugeongenesonly$gene))
rank.tPC1.from_coppaFISH <- c()
for(thisgene in tPC1.bugeon2022[tPC1.bugeon2022 %in% tPC1_fromTasic2018.bugeongenesonly$gene]){
    position_in_rnaseqPC1 <- match(thisgene, tPC1_fromTasic2018.bugeongenesonly$gene)
    rank.tPC1.from_coppaFISH <- c(rank.tPC1.from_coppaFISH, position_in_rnaseqPC1 ) # append position in other list
}

tPC1_fromTasic2018.bugeongenesonly$rank_in_tasic <- rank.tPC1.from_rnaseq
tPC1_fromTasic2018.bugeongenesonly$rank_in_bugeon <- rank.tPC1.from_coppaFISH
tPC1_fromTasic2018.bugeongenesonly$gene <- as.factor(tPC1_fromTasic2018.bugeongenesonly$gene)
rownames(tPC1_fromTasic2018.bugeongenesonly)

In [ ]:
head(tPC1_fromTasic2018.bugeongenesonly)

In [ ]:
cor.test( tPC1_fromTasic2018.bugeongenesonly$rank_in_tasic,  tPC1_fromTasic2018.bugeongenesonly$rank_in_bugeon, 
         method = 'spearman', data = tPC1_fromrnaseq.bugeongenes)

We can also compute it with just the Bugeon genes but it's better this way.

### Graded IEG expression along this axis.

In [ ]:
iegs.tasic2018.tPC1_ranking <- tPC1_fromTasic2018[tPC1_fromTasic2018$gene %in% meta.iegs,]
iegs.tasic2018.tPC1_ranking

#### Tasic v1 Inihbitory Cells Activity (Sum of all metaIEGs after normalization, including ones not computed in PC)

In [ ]:
# plotting IEG acitivy per cluster type
macthed.iegs <- rownames(tasic2018.v1_inhibitory.exon.counts.normed.unfiltered)[rownames(tasic2018.v1_inhibitory.exon.counts.normed.unfiltered) %in% meta.iegs]
tasic2018v1_inhibitory.meta$iegsum_activity <- colSums(tasic2018.v1_inhibitory.exon.counts.normed.unfiltered[macthed.iegs ,])

In [ ]:
p <- tasic2018v1_inhibitory.meta %>% ggplot(aes(x=cell_subclass, y=iegsum_activity)) + 
  geom_violin(trim=FALSE) +
  geom_point()+
  xlab('Cell Type')+
  ylab('Tasic V1 GABAergic IEG Expression Sum')+
  theme_classic()
p


In [ ]:
p <- tasic2018v1_inhibitory.meta %>% ggplot(aes(x=cell_subclass, y=iegsum_activity)) + 
  geom_violin(trim=FALSE) +
  geom_point()+
  xlab('Cell Type')+
  ylab('Tasic V1 GABAergic IEG Expression Sum')+
  theme_classic()
p

Heat map of cells activity. Cells organized by tPC1 loadings (rotation) and IEGs by PC1 scores.  Only iegs from the metaIEGl lsit in teh top 3k variable genes are included in the display, other gene were removed for computing PCA.

In [ ]:
# adding this to meta to organize cells
tasic2018v1_inhibitory.meta$tPC1_score <- tasic2018.v1_inhibitory.exon.pca$loadings$PC1

# construct dataframe from this pc score
tPC1cell.idx.df <- tasic2018v1_inhibitory.meta %>% 
                   select(sample_name, tPC1_score, iegsum_activity, cell_class, cell_subclass, cell_cluster)  %>% 
                   arrange(tPC1_score)

tPC1cell.idx.df$cell_class <- as.factor(tPC1cell.idx.df$cell_class)
tPC1cell.idx.df$cell_subclass <- as.factor(tPC1cell.idx.df$cell_subclass)
tPC1cell.idx.df$cell_cluster <- as.factor(tPC1cell.idx.df$cell_cluster)

In [ ]:
head(tPC1cell.idx.df)

In [ ]:
iegsbytpc1.tasicv1gabaergic.normed.sclaed.top3kgenes <- tasic2018.v1_inhibitory.exon.counts.normed[tPC1cell.idx.df$sample_name,iegs.tasic2018.tPC1_ranking$gene]
print(iegs.tasic2018.tPC1_ranking$gene[1:5])
head(iegsbytpc1.tasicv1gabaergic.normed.sclaed.top3kgenes)

In [ ]:
#tPC1cell.idx.df

#        annotation_row = my_sample_row)
#annot <- tPC1cell.idx.df %>% select(cell_subclass, cell_cluster)
annot <- tPC1cell.idx.df %>% select(cell_class, cell_subclass)
#annot <- as.data.frame(annot))
rownames(annot) <- tPC1cell.idx.df$sample_name

In [ ]:
temp_hm <-pheatmap(iegsbytpc1.tasicv1gabaergic.normed.sclaed.top3kgenes,
                   cluster_rows = FALSE, cluster_cols = FALSE, show_rownames = F,
                   main = "IEGs by tPC1 Tasic V1",
                   annotation_row = annot)

In [ ]:
# doing it with cell cluster means
tPC1cell.idx.df

In [ ]:
iegsbytpc1.tasicv1gabaergic.normed.sclaed.top3kgenes$Subtype <- tPC1cell.idx.df$cell_cluster
iegsbytpc1.tasicv1gabaergic.normed.sclaed.top3kgenes$tPC1_score <- tPC1cell.idx.df$tPC1_score
iegsbytpc1.tasicv1gabaergic.normed.sclaed.top3kgenes$iegsum_activity <- tPC1cell.idx.df$iegsum_activity

In [ ]:
colnames(iegsbytpc1.tasicv1gabaergic.normed.sclaed.top3kgenes)

In [ ]:
iegsbytpc1.subtype_means.df <- iegsbytpc1.tasicv1gabaergic.normed.sclaed.top3kgenes %>%
                               select(-Subtype) %>%
                               aggregate(list(iegsbytpc1.tasicv1gabaergic.normed.sclaed.top3kgenes$Subtype), mean) %>%
                               arrange(tPC1_score) %>%
                               column_to_rownames(var ='Group.1')

In [ ]:
head(iegsbytpc1.subtype_means.df)

In [ ]:
tPC1_mean.df <- data.frame( tPC1_meanscore = iegsbytpc1.subtype_means.df$tPC1_score,
                            iegsum_meanactivity = iegsbytpc1.subtype_means.df$iegsum_activity)
rownames(tPC1_mean.df) <- rownames(iegsbytpc1.subtype_means.df)
tPC1_mean.df$relative.scaled.means <- scale(tPC1_mean.df$tPC1_meanscore)
tPC1_mean.df$relative.scaled.iegsum_meanactivity  <- scale(tPC1_mean.df$iegsum_meanactivity)

In [ ]:
subtype_to_type_annotation <- function(subtype_string, list_of_subtypes, list_of_types){
                                #checks the strings of subtypes to for begingin string
                                #to map to a type, ends loop if one is detected returns 
                                # what it was given, returns match if it finds something 
                                # and en empty string if there is no match
                                # includes a break stement to stop loop once match is found
                                # assumes list_of_subtypes and list_of_types are same length
                                # with corresponging entries
                                output.string <- ""
                                for(i in c(1:length(list_of_subtypes))){
                                if(startsWith(subtype_string, list_of_subtypes[i])){
                                   output.string <- list_of_types[i]
                                   break
                                    }# end of ifstatement
                                  }# end of for loop
                                return(output.string)
                                }# end of function

In [ ]:
subclass.names <- unique(tasic2018v1_inhibitory.meta$cell_subclass)
subclass.names

In [ ]:
tPC1_mean.df$subclass <- sapply(rownames(tPC1_mean.df), FUN = function(x) subtype_to_type_annotation(x, types.names, types.names) )

In [ ]:
iegsbytpc1.subtype_means.df <- select(iegsbytpc1.subtype_means.df, -tPC1_score)
iegsbytpc1.subtype_means.df <- select(iegsbytpc1.subtype_means.df, -iegsum_activity)

In [ ]:
head(tPC1_mean.df)

In [ ]:
subclass.df <- data.frame('Subclass'=as.factor(tPC1_mean.df$subclass))
rownames(subclass.df) <- rownames(iegsbytpc1.subtype_means.df)

In [ ]:
library(dichromat)

In [ ]:
mycolors <- c('#4daf4a','#377eb8','#e41a1c','#984ea3','#ff7f00')
names(mycolors) <- unique(subclass.df$Subclass)
mycolors <- list(mycolors = mycolors)
mycolors

In [ ]:
#annotation_colors = mycolors,

temp_hm <-pheatmap(iegsbytpc1.subtype_means.df,
                   cluster_rows = FALSE, cluster_cols = FALSE,
                   annotation_row = subclass.df,
                   main = "IEGs by tPC1 Tasic V1")

In [ ]:
save_pheatmap(temp_hm, filename='Tasic_tPC1_V1_GABAergic_meanIEGexpression.png', width = 600, height=600)

In [ ]:
tasic2018v1_inhibitory.meta$scaled.tPC1_score <- scale(tasic2018v1_inhibitory.meta$tPC1_score)
tasic2018v1_inhibitory.meta$scaled.iegsum_activity <- scale(tasic2018v1_inhibitory.meta$iegsum_activity)

In [ ]:
#we use the tPC1_mean.df to order the elvels in the way we want
# note we had to use the rev function to get the exact order we want
tasic2018v1_inhibitory.meta$cell_cluster <- factor(tasic2018v1_inhibitory.meta$cell_cluster, levels=rev(rownames(tPC1_mean.df)) )

In [ ]:
p <- tasic2018v1_inhibitory.meta %>%  
     ggplot(aes(x=cell_cluster, y=scaled.tPC1_score)) + 
     geom_boxplot() +
     geom_point()+
     coord_flip()+
     xlab('Cell Subtype (Cluster)')+
     ylab('Tasic V1 GABAergic tPC1 score')+
     theme_classic()
p

In [ ]:
p <- tasic2018v1_inhibitory.meta %>%  
     ggplot(aes(x=cell_cluster, y=scaled.iegsum_activity)) + 
     geom_boxplot() +
     geom_point()+
     coord_flip()+
     xlab('Cell Subtype (Cluster)')+
     ylab('Tasic V1 GABAergic tPC1 score')+
     theme_classic()
p

In [ ]:
# boot strap modulatiry scores for multiple genes
# compute the activity score for each cell type as a percentage
# of significant gradient genes, significantly modular genes and 
# genes that are both or niether

In [ ]:
#p <- bp tPC1_mean.df$relative.scaled.means

###  Full gene set we calculate significant genes in data

In [ ]:
library(tictoc)

In [ ]:
# filter low expressed genes becuase they will not be removed 

# We first removed predicted gene models (gene names that start with Gm), genes from the mitochondrial chromosome, 
# ribosomal genes, sex-specific genes, as well as genes that were detected in fewer than four cells.

In [ ]:
dim(tasic2018.v1_inhibitory.exon.counts)
dim(tasic2018.v1_inhibitory.exon.counts.normed.unfiltered)

In [ ]:
# remove gene model genes
filt.genes.idx <- !startsWith( rownames(tasic2018.v1_inhibitory.exon.counts.normed.unfiltered), 'Gm') 
# remove genes expressed in less than 4 cells
filt.genes.idx <- filt.genes.idx & ( rowSums(tasic2018.v1_inhibitory.exon.counts[matched.protcoding.genes,]>0) > 4 )

# checks length of genes we can see ~3k are removed based on above criteria from Tasic
length(filt.genes.idx)
sum(filt.genes.idx)
filt.genes.idx[1:10]
names(filt.genes.idx)[filt.genes.idx==TRUE][1:10]

In [ ]:
tasic2018.v1_inhibitory.exon.counts.normed.allgenes <-as.data.frame(scale(t(tasic2018.v1_inhibitory.exon.counts.normed.unfiltered[filt.genes.idx,])))
# after t() the data becomes a matrix and loses its colnames an rownames we need to get them from the og df but row ->cols and vice versa
genes <- rownames(tasic2018.v1_inhibitory.exon.counts.normed.unfiltered)[filt.genes.idx]
colnames(tasic2018.v1_inhibitory.exon.counts.normed.allgenes) <- genes #genes
rownames(tasic2018.v1_inhibitory.exon.counts.normed.allgenes) <- colnames(tasic2018.v1_inhibitory.exon.counts.normed.unfiltered) #cellids

In [ ]:
dim(tasic2018.v1_inhibitory.exon.counts.normed.unfiltered)
dim(tasic2018.v1_inhibitory.exon.counts.normed.allgenes)

In [ ]:
tic()
tasic2018.v1_inhibitory.prcomp.allgenes <- prcomp(tasic2018.v1_inhibitory.exon.counts.normed.allgenes, scale=FALSE)
tasic2018.v1_inhibitory.prcomp.allgenes <- pca_loading(tasic2018.v1_inhibitory.prcomp.allgenes) # computes laodings (correlation of genes with PC's) and their pvalues
toc()

In [ ]:
str(tasic2018.v1_inhibitory.prcomp.allgenes)

In [ ]:
# lets adjust for multiple corrections within a PC
tPC1_gene_qvalues_BH <- p.adjust(tasic2018.v1_inhibitory.prcomp.allgenes$loading$p.value[,1], method='BH')
sum(tPC1_gene_qvalues_BH<0.01)

In [ ]:
# now we pull out the IEGs which are significantly correlated with tPC1
fdr.thresh <- 0.01

length(meta.iegs[meta.iegs %in% names(tPC1_gene_qvalues_BH)[tPC1_gene_qvalues_BH<fdr.thresh]])
meta.iegs.sig_on_tpc1 <- meta.iegs[meta.iegs %in% names(tPC1_gene_qvalues_BH)[tPC1_gene_qvalues_BH<fdr.thresh]]
print(meta.iegs.sig_on_tpc1)
tPC1_gene_qvalues_BH[names(tPC1_gene_qvalues_BH) %in% meta.iegs.sig_on_tpc1]

In [ ]:
tPC1_fromTasic2018 <- data.frame(gene = colnames(tasic2018.v1_inhibitory.exon.counts.normed.allgenes),
                                           tPC1 = tasic2018.v1_inhibitory.prcomp.allgenes$loading$R[,1],
                                 p.value = tasic2018.v1_inhibitory.prcomp.allgenes$loading$p.value[,1],
                                q.value_BH = p.adjust(tasic2018.v1_inhibitory.prcomp.allgenes$loading$p.value[,1], method='BH')) %>% arrange(tPC1)
head(tPC1_fromTasic2018)

In [ ]:
fdr.thresh <- 0.01
iegs.tasic2018.tPC1_ranking <- tPC1_fromTasic2018[(tPC1_fromTasic2018$q.value_BH<fdr.thresh)&(tPC1_fromTasic2018$gene %in% meta.iegs),]
dim(iegs.tasic2018.tPC1_ranking)
head(iegs.tasic2018.tPC1_ranking )

In [ ]:
tasic2018v1_inhibitory.meta$tPC1_score <- tasic2018.v1_inhibitory.prcomp.allgenes$x[,1]

# construct dataframe from this pc score
tPC1cell.idx.df <- tasic2018v1_inhibitory.meta %>% 
                   select(sample_name, tPC1_score, iegsum_activity, cell_class, cell_subclass, cell_cluster)  %>% 
                   arrange(tPC1_score)

tPC1cell.idx.df$cell_class <- as.factor(tPC1cell.idx.df$cell_class)
tPC1cell.idx.df$cell_subclass <- as.factor(tPC1cell.idx.df$cell_subclass)
tPC1cell.idx.df$cell_cluster <- as.factor(tPC1cell.idx.df$cell_cluster)

In [ ]:
head(tPC1cell.idx.df)

In [ ]:
head(tasic2018.v1_inhibitory.exon.counts.normed)

In [ ]:
#as.character(tPC1cell.idx.df$sample_name)

In [ ]:
sum(iegs.tasic2018.tPC1_ranking$gene %in% rownames(tasic2018.v1_inhibitory.exon.counts.normed.unfiltered))

In [ ]:
iegsbytpc1.tasicv1gabaergic.normed.scaled <- tasic2018.v1_inhibitory.exon.counts.normed.unfiltered[tPC1cell.idx.df$sample_name, iegs.tasic2018.tPC1_ranking$gene]
print(iegs.tasic2018.tPC1_ranking$gene[1:5])
head(iegsbytpc1.tasicv1gabaergic.normed.scaled)

### PCA full compoenent significance testing by boostrapping

In [ ]:
#write.csv('tasci2018_V1_gaba_protcodinggenes_normed.csv',tasic2018.v1_inhibitory.exon.counts.normed.unfiltered, row.names=FALSE)

In [ ]:
# it's scaling the data which is causing bugs
tic()
pca.allgenes.test <- PCAtest(tasic2018.v1_inhibitory.exon.counts.normed.allgenes, 
                                                   100, 100, 0.05, varcorr=FALSE, counter=FALSE, plot=TRUE)
toc()

In [ ]:
which(colSums(tasic2018.v1_inhibitory.exon.counts.normed.allgenes)==0)

In [ ]:
sum(rowSums(tasic2018.v1_inhibitory.exon.counts.normed.unfiltered[filt.genes.idx,])==0)

In [ ]:
rowSums(tasic2018.v1_inhibitory.exon.counts[c('Ptger3','Tnfsf15'),])

In [ ]:
rowSums(tasic2018.v1_inhibitory.exon.counts[c('Ptger3','Tnfsf15'),]>10)

In [ ]:
rowMeans(tasic2018.v1_inhibitory.exon.counts[c('Ptger3','Tnfsf15'),])

In [ ]:
rowMedians(tasic2018.v1_inhibitory.exon.counts[c('Ptger3','Tnfsf15'),])

In [ ]:
head(tasic2018.v1_inhibitory.exon.counts[c('Ptger3','Tnfsf15'),])

### Tasic again with just the bugeon genes

In [ ]:
# can be pulled from the OG coutns datasets we just need to re run the scaling or whatever
tasic2018.v1_inhibitory.pca.restrictedlist <- pca(tasic2018.v1_inhibitory.exon.counts.normed[,bugeon2022.genes])
plot(tasic2018.v1_inhibitory.pca.restrictedlist$variance)

In [ ]:
tPC1_fromTasic2018.bugeongenesonly <- data.frame(gene = bugeon2022.genes,
                                           tPC1 = tasic2018.v1_inhibitory.pca.restrictedlist$rotated$PC1) %>% arrange(tPC1)
    
print(dim(tPC1_fromTasic2018.bugeongenesonly  ))
tPC1_fromTasic2018.bugeongenesonly

In [ ]:
# I would like to point out Rgs4 is also an IEG, shows up as important in active dentate gyrus cells as well

In [ ]:
# careful here PCA can flip signs, just make sure Pvalb and Vip are at opposing ends of the rankings
rank.tPC1.from_rnaseq <- c(1:length(tPC1_fromTasic2018.bugeongenesonly$gene))
rank.tPC1.from_coppaFISH <- c()
for(thisgene in tPC1.bugeon2022){
    position_in_rnaseqPC1 <- match(thisgene, tPC1_fromTasic2018.bugeongenesonly$gene)
    rank.tPC1.from_coppaFISH <- c(rank.tPC1.from_coppaFISH, position_in_rnaseqPC1 ) # append position in other list
}

tPC1_fromTasic2018.bugeongenesonly$rank_in_tasic <- rank.tPC1.from_rnaseq
tPC1_fromTasic2018.bugeongenesonly$rank_in_bugeon <- rank.tPC1.from_coppaFISH
tPC1_fromTasic2018.bugeongenesonly$gene <- as.factor(tPC1_fromTasic2018.bugeongenesonly$gene)
rownames(tPC1_fromrnaseq.bugeongenes)

In [ ]:
head(tPC1_fromTasic2018.bugeongenesonly)

In [ ]:
cor.test( tPC1_fromTasic2018.bugeongenesonly$rank_in_tasic,  tPC1_fromTasic2018.bugeongenesonly$rank_in_bugeon, 
         method = 'spearman', data = tPC1_fromrnaseq.bugeongenes)

In [ ]:
# plot cells average acitivity score across the clusters, show some are more active
# plot the individual cells IEG expression vs their rank along the gradient
#, rank the IEGs by their position on tPC1, consider restricting to protien coding genes
# plot the clusters averag IEG expression again IEGs ordered by their position on PC1 and clusters ordered by their cells average
# position on PC1 posibly just using the bugeon genes